In [1]:
!pip install langchain

**Explanation:** Installs core LangChain package (plus dependencies pulled transitively). We keep versions implicit here to stay current; pin in production for reproducibility.

# LangChain Memory Classes – Practical Comparison

This notebook provides a hands-on tour of multiple LangChain conversational memory types:

- ConversationBufferMemory (full running transcript)
- ConversationBufferWindowMemory (sliding window)
- ConversationEntityMemory (entity-centric fact retention)
- (Underlying mechanics: save/load context, message objects, prompt integration)

Goals:
1. Understand when each memory pattern is appropriate (cost, context length, personalization).
2. Inspect internal structures (`chat_memory.messages`, `entity_store.store`).
3. Demonstrate multi-turn querying and selective recall.

Security: Any API keys (e.g., Gemini) must be supplied via environment variables or notebook secret helpers. Placeholders are used; replace locally. Each executable cell should be followed by an explanatory markdown block (being added in this polish pass).

In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


**Explanation:** Installs community and Google Generative AI integration packages (langchain-community, langchain_google_genai). Split install helps show optional provider-specific layer.

In [3]:
!pip install langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


**Explanation:** Adds the Google Generative AI (Gemini) LangChain integration. Required only if you plan to use Gemini models; keep optional in lean environments.

In [4]:
import warnings
warnings.filterwarnings('ignore')

**Explanation:** Suppresses non-critical warnings for cleaner educational output. Avoid in debugging scenarios where warnings may indicate deprecations.

In [ ]:
import os
# --- Secure API Key Handling ---
# Provide your Gemini API key via environment variable or a secret manager.
# DO NOT hardcode secrets in shared notebooks.
# Example (uncomment and replace placeholder locally):
# os.environ["GOOGLE_API_KEY"] = "WRITE_YOUR_OWN_KEY"

# If running in Google Colab with stored secrets, you can optionally use:
# from google.colab import userdata
# os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

**Explanation:** Retrieves the Gemini API key from Colab secret storage and sets environment variable `GOOGLE_API_KEY`. Replace with your own secure secret management if running locally (e.g., `os.environ['GOOGLE_API_KEY'] = 'WRITE_YOUR_OWN_KEY'`).

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite-001",convert_system_message_to_human=True)

**Explanation:** Creates a Gemini chat model wrapper. `convert_system_message_to_human=True` adapts system instructions for models that expect only user/assistant roles. Adjust model name to the latest available variant if needed.

In [8]:
print(model.invoke("hi").content)

Hi there! How can I help you today?


**Explanation:** Sanity-checks the LLM pipeline by sending a minimal greeting; validates credentials and network before building memory abstractions.

In [9]:
print(model.invoke("hi, how are you please tell me?").content)

I am doing well, thank you for asking! As a large language model, I don't experience feelings like humans do, but I am functioning properly and ready to assist you. How can I help you today?


**Explanation:** Sends a slightly longer conversational prompt to observe style/latency differences vs a short token test; still before memory comes into play.

In [10]:
from langchain.memory import ConversationBufferMemory

**Explanation:** Imports the simplest memory type which stores the entire running transcript as a growing string (or list when `return_messages=True`). Use for short sessions.

In [11]:
memory = ConversationBufferMemory()

/tmp/ipython-input-995385594.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


**Explanation:** Instantiates a buffer memory (string return). Default keys: `input` and `output`. Grows unbounded—mind token limits.

In [12]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})

**Explanation:** Manually appends a user/AI exchange to the transcript. Useful for simulating prior turns before starting a chain.

In [13]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

**Explanation:** Retrieves the aggregated context dict; for buffer memory when `return_messages=False` value is a concatenated string under the default memory key (`history`).

In [14]:
memory2 = ConversationBufferMemory(return_messages=True)

**Explanation:** Second buffer memory instance configured with `return_messages=True` so downstream chains receive a structured list of `HumanMessage` / `AIMessage` objects instead of a joined string.

In [15]:
memory2.save_context({"input": "Hi"},
                    {"output": "What's up"})

**Explanation:** Appends a turn to `memory2` (messages list). This structured form enables downstream tools to inspect roles and segment messages cleanly.

In [16]:
memory2.load_memory_variables({})

{'history': [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content="What's up", additional_kwargs={}, response_metadata={})]}

**Explanation:** Returns a dict whose `history` value is now a list of message objects (since `return_messages=True`). Useful for tool-aware chains.

In [17]:
import os

**Explanation:** Imports `os` (already imported earlier) – retained from original notebook; could be removed without impact.

In [18]:
from langchain.chains import ConversationChain

**Explanation:** Imports high-level `ConversationChain` which stitches together an LLM, a prompt template (implicit default), and memory to produce contextual responses.

In [19]:
conversation = ConversationChain(llm=model,verbose=True,memory=ConversationBufferMemory())

/tmp/ipython-input-2263270236.py:1: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(llm=model,verbose=True,memory=ConversationBufferMemory())


**Explanation:** Constructs a `ConversationChain` using the simple buffer memory; `verbose=True` will show internal prompt assembly—helpful for learning.

In [20]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


"Hello there! It's a pleasure to chat with you today. I'm ready to answer your questions, discuss various topics, or just have a friendly conversation. I have access to a vast amount of information, so feel free to ask me anything. I'm particularly knowledgeable about history, science, literature, and technology, but I'm always learning and happy to explore new areas. So, what's on your mind? I'm eager to hear from you!"

**Explanation:** First turn through `ConversationChain`; memory initially empty so only this user input influences the prompt.

In [21]:
conversation.predict(input="Nothing much! Just tell me how do a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello there! It's a pleasure to chat with you today. I'm ready to answer your questions, discuss various topics, or just have a friendly conversation. I have access to a vast amount of information, so feel free to ask me anything. I'm particularly knowledgeable about history, science, literature, and technology, but I'm always learning and happy to explore new areas. So, what's on your mind? I'm eager to hear from you!
Human: Nothing much! Just tell me how do a conversation with an AI.
AI:

> Finished chain.


'Well, that\'s a great question! Having a conversation with me, an AI, is pretty straightforward, and there are a few things to keep in mind to make it a productive and enjoyable experience. Here\'s a breakdown:\n\n*   **Start with a Greeting:** Just like you did! A simple "Hi there," "Hello," or "Good day" is a perfect way to begin. It signals that you\'re ready to interact.\n\n*   **Ask Clear Questions or State Your Topic:** The more specific you are, the better I can understand your needs. Instead of just saying "Tell me about space," you could ask, "What are the latest discoveries about black holes?" or "What are the different types of space telescopes?" This helps me focus my response.\n\n*   **Be Patient:** Sometimes, I might take a few moments to process your request, especially if it\'s complex. Don\'t worry, I\'m working hard behind the scenes!\n\n*   **Provide Context:** If you\'re asking a follow-up question, it\'s helpful to remind me of the previous conversation. This help

**Explanation:** Second turn—prior assistant reply now included in prompt via buffer memory, enabling continuity when giving meta-advice about AI conversations.

In [22]:
conversation.predict(input="how many tips are there can you mention in the numbers")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello there! It's a pleasure to chat with you today. I'm ready to answer your questions, discuss various topics, or just have a friendly conversation. I have access to a vast amount of information, so feel free to ask me anything. I'm particularly knowledgeable about history, science, literature, and technology, but I'm always learning and happy to explore new areas. So, what's on your mind? I'm eager to hear from you!
Human: Nothing much! Just tell me how do a conversation with an AI.
AI: Well, that's a great question! Having a conversation with me, an AI, is pretty straightforward, and there are a few things to keep in mind to make it a productiv

'Certainly! In my previous response, I provided a list of tips on how to have a conversation with me. Let\'s count them:\n\n1.  **Start with a Greeting:** (e.g., "Hi there," "Hello")\n2.  **Ask Clear Questions or State Your Topic:** Be specific.\n3.  **Be Patient:** I might take a moment to process.\n4.  **Provide Context:** Remind me of the previous conversation.\n5.  **Use Natural Language:** Speak to me as you would to a person.\n6.  **Be Open to My Responses:** I\'ll provide accurate information.\n7.  **Ask for Clarification if Needed:** If something isn\'t clear, ask.\n8.  **Be Mindful of Your Tone:** Be polite and respectful.\n9.  **Have Fun!** Enjoy the conversation.\n\nSo, there are a total of **9** tips mentioned in the previous response. Is there anything else I can help you with?'

**Explanation:** Queries for a numeric summary (tip count) relying on previous assistant output still present in full buffer.

In [23]:
conversation.memory.chat_memory.messages

[HumanMessage(content='Hi there!', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello there! It's a pleasure to chat with you today. I'm ready to answer your questions, discuss various topics, or just have a friendly conversation. I have access to a vast amount of information, so feel free to ask me anything. I'm particularly knowledgeable about history, science, literature, and technology, but I'm always learning and happy to explore new areas. So, what's on your mind? I'm eager to hear from you!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Nothing much! Just tell me how do a conversation with an AI.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Well, that\'s a great question! Having a conversation with me, an AI, is pretty straightforward, and there are a few things to keep in mind to make it a productive and enjoyable experience. Here\'s a breakdown:\n\n*   **Start with a Greeting:** Just like you did! A simple "Hi th

**Explanation:** Directly inspects internal `chat_memory.messages` list to show accumulated Human/AI message objects—useful for debugging prompt size.

In [24]:
conversation.predict(input="can you give me the 3rd tip?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello there! It's a pleasure to chat with you today. I'm ready to answer your questions, discuss various topics, or just have a friendly conversation. I have access to a vast amount of information, so feel free to ask me anything. I'm particularly knowledgeable about history, science, literature, and technology, but I'm always learning and happy to explore new areas. So, what's on your mind? I'm eager to hear from you!
Human: Nothing much! Just tell me how do a conversation with an AI.
AI: Well, that's a great question! Having a conversation with me, an AI, is pretty straightforward, and there are a few things to keep in mind to make it a productiv

'Absolutely! The 3rd tip I mentioned was:\n\n**3. Be Patient:** I might take a moment to process.\n\nThis means that sometimes, especially when dealing with complex questions or requests, I might need a few seconds to gather information and formulate a response. So, a little patience is appreciated! Is there anything else I can assist you with?'

**Explanation:** Asks for the 3rd tip—tests ability of buffer memory to retain earlier enumerated advice and reference by ordinal.

# ConversationBufferWindowMemory

In [25]:
from langchain.memory import ConversationBufferWindowMemory

**Explanation:** Imports sliding window memory variant which only keeps the last `k` exchanges to control prompt growth and cost.

In [26]:
window_memory = ConversationBufferWindowMemory(k=1)

/tmp/ipython-input-1930410100.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  window_memory = ConversationBufferWindowMemory(k=1)


**Explanation:** Creates a window memory retaining only the most recent 1 exchange pair (user+AI). Earlier turns are dropped—useful for minimizing prompt size while preserving immediate context.

In [27]:
window_memory.save_context(
    {"input": "Hi"},
    {"output": "What's up"}
)

**Explanation:** Adds first exchange to window memory (k=1). Subsequent additions will evict older turns beyond the most recent pair.

In [28]:
window_memory.save_context(
    {"input": "Not much, just hanging"},
    {"output": "Cool"}
)

**Explanation:** Adds a second exchange. With `k=1`, this now evicts the earlier turn, keeping only the most recent conversational pair.

In [29]:
window_memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

**Explanation:** Shows the current retained context (only last 1 exchange because `k=1`). Older turns are intentionally absent.

In [30]:
window_memory = ConversationBufferWindowMemory( k=2, return_messages=True)

**Explanation:** Reinitializes window memory to keep the last 2 exchanges (messages list form). Balances context retention with bounded growth.

In [31]:
window_memory.save_context(
    {"input": "Hi"},
    {"output": "What's up"}
)
window_memory.save_context(
    {"input": "Not much, just hanging"},
    {"output": "Cool"}
)
window_memory.save_context(
    {"input": "ok thanks "},
    {"output": "great thankyou"}
)


**Explanation:** Adds three sequential exchanges; with `k=2` only the last two exchanges are retained. Demonstrates eviction as new turns arrive.

In [32]:
window_memory.load_memory_variables({})

{'history': [HumanMessage(content='Not much, just hanging', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Cool', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='ok thanks ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='great thankyou', additional_kwargs={}, response_metadata={})]}

**Explanation:** Loads current retained context (last two exchanges only) after multiple additions—verifies eviction logic for `k=2`. 

In [33]:
conversation_window = ConversationChain(
    llm=model,
    memory=window_memory,
    verbose=True
)

**Explanation:** Builds a `ConversationChain` wired to the `window_memory`. Only the last `k` exchanges (currently 2) will inform responses—helps simulate limited context scenarios.

In [34]:
conversation_window.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Not much, just hanging', additional_kwargs={}, response_metadata={}), AIMessage(content='Cool', additional_kwargs={}, response_metadata={}), HumanMessage(content='ok thanks ', additional_kwargs={}, response_metadata={}), AIMessage(content='great thankyou', additional_kwargs={}, response_metadata={})]
Human: Hi, what's up?
AI:

> Finished chain.


"Hi there! Not much on my end, just processing information and keeping the vast knowledge base updated. I'm currently analyzing a collection of text documents about various topics, including the history of the Roman Empire, the intricacies of quantum physics, and the latest advancements in artificial intelligence, of course! I'm also cross-referencing information to identify potential connections and contradictions. It's a pretty busy, but fascinating, day! How about you? What's keeping you busy?"

**Explanation:** First turn in window-based chain; establishes context baseline before eviction dynamics occur.

In [35]:
conversation_window.predict(input="how we can talk with AI give me 5 points")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='ok thanks ', additional_kwargs={}, response_metadata={}), AIMessage(content='great thankyou', additional_kwargs={}, response_metadata={}), HumanMessage(content="Hi, what's up?", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi there! Not much on my end, just processing information and keeping the vast knowledge base updated. I'm currently analyzing a collection of text documents about various topics, including the history of the Roman Empire, the intricacies of quantum physics, and the latest advancements in artificial intelligence, of course! I'm also cross-referencing information to identify potential connections and contradic

'Okay, here are 5 points on how you can effectively communicate with me (or other AI models):\n\n1.  **Be Clear and Specific:** The more precise you are in your questions or prompts, the better I can understand your needs. Avoid vague language or ambiguous phrasing. For example, instead of asking "Tell me about science," try "Explain the theory of relativity in simple terms." This helps me narrow down the scope and provide a more relevant response.\n\n2.  **Break Down Complex Questions:** If you have a complicated question, try breaking it down into smaller, more manageable parts. This helps me process the information step by step. You can ask follow-up questions to clarify each part. For instance, if you want to know about the impact of the printing press, you could ask: "What was the printing press?" followed by "When was it invented?" and then "What were its impacts on society?"\n\n3.  **Provide Context:** Giving me context can significantly improve my responses. Let me know what yo

**Explanation:** Adds a content-heavy request (5 points) which will later test whether older enumerated content survives as window rolls forward.

In [36]:
conversation_window.predict(input="what is a allows AI to 'see' and 'interpret' images?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content="Hi, what's up?", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi there! Not much on my end, just processing information and keeping the vast knowledge base updated. I'm currently analyzing a collection of text documents about various topics, including the history of the Roman Empire, the intricacies of quantum physics, and the latest advancements in artificial intelligence, of course! I'm also cross-referencing information to identify potential connections and contradictions. It's a pretty busy, but fascinating, day! How about you? What's keeping you busy?", additional_kwargs={}, response_metadata={}), HumanMessage(content='how

'That\'s a great question! What allows AI to "see" and "interpret" images is a field called **computer vision**. It\'s a fascinating area of artificial intelligence that focuses on enabling computers to "understand" images and videos in a way that\'s similar to how humans do.\n\nHere\'s a bit more detail about how it works:\n\n*   **Image Input:** It all starts with an image, which can be a photograph, a video frame, or even a drawing. The image is essentially a grid of pixels, each with a color value.\n*   **Feature Extraction:** The AI uses various algorithms to analyze the image and extract meaningful features. These features can include edges, corners, textures, and shapes. This is often done using techniques like convolutional neural networks (CNNs), which are specifically designed to process visual data.\n*   **Object Recognition:** The AI then uses these features to identify objects within the image. This involves comparing the extracted features to a database of known objects. 

**Explanation:** Asks a knowledge-style question unrelated to prior tips—tests model response when context offers little direct relevance; window still carries last two exchanges only.

In [37]:
conversation_window.predict(input="can you tell me how many tips you genearte in the previous to previous message?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='how we can talk with AI give me 5 points', additional_kwargs={}, response_metadata={}), AIMessage(content='Okay, here are 5 points on how you can effectively communicate with me (or other AI models):\n\n1.  **Be Clear and Specific:** The more precise you are in your questions or prompts, the better I can understand your needs. Avoid vague language or ambiguous phrasing. For example, instead of asking "Tell me about science," try "Explain the theory of relativity in simple terms." This helps me narrow down the scope and provide a more relevant response.\n\n2.  **Break Down Complex Questions:** If you have a complicated question, try breaking it do

'Certainly! In the previous-to-previous message, I generated 5 tips on how to communicate effectively with AI. They were:\n\n1.  Be Clear and Specific\n2.  Break Down Complex Questions\n3.  Provide Context\n4.  Use Keywords\n5.  Iterate and Refine'

**Explanation:** Tests recall of a value (tip count) that may have been evicted if original enumeration not within last `k` exchanges—illustrates limitation of windowed memory.

In [38]:
conversation.predict(input="what was the fifth number tips?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello there! It's a pleasure to chat with you today. I'm ready to answer your questions, discuss various topics, or just have a friendly conversation. I have access to a vast amount of information, so feel free to ask me anything. I'm particularly knowledgeable about history, science, literature, and technology, but I'm always learning and happy to explore new areas. So, what's on your mind? I'm eager to hear from you!
Human: Nothing much! Just tell me how do a conversation with an AI.
AI: Well, that's a great question! Having a conversation with me, an AI, is pretty straightforward, and there are a few things to keep in mind to make it a productiv

"The fifth tip I mentioned was:\n\n**5. Use Natural Language:** Speak to me as you would to a person.\n\nEssentially, you don't need to use any special commands or formatting. Just talk to me as you would to another human, and I'll do my best to understand! Is there anything else I can help you with?"

**Explanation:** Uses the full buffer chain (not window) to retrieve a specific ordinal tip—succeeds because full history still present.

In [39]:
from langchain.memory import ConversationEntityMemory

**Explanation:** Imports entity-centric memory which extracts and tracks facts about named entities (people, orgs, etc.) separately from raw transcript.

In [40]:
memory = ConversationEntityMemory(llm=model)

/tmp/ipython-input-1182818481.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationEntityMemory(llm=model)
/usr/local/lib/python3.11/dist-packages/pydantic/main.py:253: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


**Explanation:** Instantiates entity memory with the LLM (used internally to extract/update entity summaries). Can increase latency due to summarization calls.

In [41]:
_input = {"input": "Deven & Sam are working on a hackathon project"}

**Explanation:** Prepares an input mentioning two entities (Deven & Sam) so the entity memory can extract and track them.

In [42]:
memory.load_memory_variables(_input)

{'history': '', 'entities': {'Deven': '', 'Sam': ''}}

**Explanation:** Loads memory variables for the first time—entity store initially empty; helps show structure pre-population.

In [43]:
memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

**Explanation:** Saves the initial exchange—LLM internally extracts entities and updates their summaries (e.g., associates Deven & Sam with a project).

In [44]:
memory.load_memory_variables({"input": 'who is Sam'})

{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?',
 'entities': {'Sam': 'Sam is working on a hackathon project.'}}

**Explanation:** Queries memory for entity info about 'Sam'—should retrieve structured summary if extraction succeeded.

In [45]:
memory = ConversationEntityMemory(llm=model, return_messages=True)

**Explanation:** Recreates entity memory returning structured message objects—beneficial when composing with other components that inspect roles.

In [46]:
memory

ConversationEntityMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, llm=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-lite-001', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7a48b22605d0>, default_metadata=(), convert_system_message_to_human=True, model_kwargs={}), entity_store=InMemoryEntityStore(store={}))

**Explanation:** Raw inspection of memory object to view its configuration/state representation (helpful for debugging fields).

In [47]:
_input = {"input": "Deven & Sam are working on a hackathon project"}

**Explanation:** Reuses initial statement to show deterministic extraction of entities on a fresh memory instance returning messages.

In [48]:
memory.load_memory_variables(_input)

{'history': [], 'entities': {'Deven': '', 'Sam': ''}}

**Explanation:** Loads variables (entity store still empty prior to save). Mirrors earlier non-return_messages path but with structured messages.

In [49]:
memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

**Explanation:** Persists first exchange; triggers extraction & summarization step populating the entity store with Deven and Sam entries.

In [50]:
memory.load_memory_variables({"input": 'who is Sam'})

{'history': [HumanMessage(content='Deven & Sam are working on a hackathon project', additional_kwargs={}, response_metadata={}),
  AIMessage(content=' That sounds like a great project! What kind of project are they working on?', additional_kwargs={}, response_metadata={})],
 'entities': {'Sam': 'Sam is working on a hackathon project.'}}

**Explanation:** Retrieves updated summary for Sam from entity store (should now include project participation details).

In [51]:
from langchain.chains import ConversationChain

**Explanation:** Re-import (already present earlier); retained for clarity in isolated notebook sections—could be removed when refactoring.

In [52]:
from langchain.memory import ConversationEntityMemory

**Explanation:** Re-import for entity memory to emphasize its use in a custom prompt chain section.

In [53]:
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

**Explanation:** Imports built-in prompt template tailored for entity memory injection—includes slots for entity summaries and conversation history.

In [54]:
from pydantic import BaseModel

**Explanation:** Imports Pydantic `BaseModel` (not directly used later in this notebook—kept from original; could be removed).

In [55]:
from typing import List, Dict, Any

**Explanation:** Imports typing utilities (also unused here)—candidates for cleanup during refactor.

In [56]:
conversation = ConversationChain(
    llm=model,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=model)
)

**Explanation:** Builds a conversation chain that injects entity summaries into a specialized prompt, enabling targeted recall about previously mentioned people/objects.

In [57]:
conversation.predict(input="Deven & Sam are working on a hackathon project")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"That's great! What kind of project are they working on? I'm curious to hear more about it."

**Explanation:** Establishes entity references; chain will create/update summaries for Deven and Sam on this turn.

In [58]:
conversation.memory.entity_store.store

{'Deven': 'Deven is working on a hackathon project.',
 'Sam': 'Sam is working on a hackathon project.'}

**Explanation:** Inspects internal entity store mapping entity names to evolving summaries—key artifact of entity memory.

In [59]:
conversation.predict(input="They are trying to add more complex memory structures to Langchain")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"That sounds like an interesting project! Adding more complex memory structures to Langchain could open up a lot of possibilities. I'm curious, what kind of memory structures are they exploring? Are they looking at different types of memory, like long-term or short-term, or perhaps different ways to store and retrieve information?"

**Explanation:** Adds new factual nuance ("more complex memory structures")—entity summaries may expand with project focus details.

In [60]:
conversation.predict(input="They are adding in a key-value store for entities mentioned so far in the conversation.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"That's a clever approach! Using a key-value store to track entities mentioned in the conversation within Langchain's memory is a practical way to enhance its ability to remember and reference specific information. This could significantly improve the context and coherence of the responses generated by Langchain. Are they using a specific key-value store implementation, or are they building their own?"

**Explanation:** Adds explicit description of entity memory mechanism (key-value store) reinforcing meta-awareness in conversation history.

In [61]:
conversation.predict(input="What do you know about Deven & Sam?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'Based on the context, I know that Deven and Sam are both working on a hackathon project.'

**Explanation:** Queries consolidated knowledge about both entities—response should synthesize evolving summary entries.

In [62]:
from pprint import pprint
pprint(conversation.memory.entity_store.store)

{'Deven': 'Deven is working on a hackathon project.',
 'Langchain': 'Deven and Sam are trying to add more complex memory structures '
              'to Langchain.',
 'Sam': 'Sam is working on a hackathon project.'}


**Explanation:** Pretty-prints entity store for clearer visual inspection of accumulated summaries after multiple updates.

In [63]:
conversation.predict(input="Sam is the founder of a company called Daimon.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"That's interesting! So, Sam is the founder of Daimon, and they're working on a hackathon project. It seems like they might be juggling multiple commitments. Is Daimon related to the hackathon project, or are they separate endeavors?"

**Explanation:** Injects a new attribute about Sam (founder role) — entity memory should merge this into Sam's evolving summary.

In [64]:
from pprint import pprint
pprint(conversation.memory.entity_store.store)

{'Daimon': 'Sam is the founder of Daimon.',
 'Deven': 'Deven is working on a hackathon project.',
 'Langchain': 'Deven and Sam are trying to add more complex memory structures '
              'to Langchain.',
 'Sam': 'Sam is the founder of Daimon.'}


**Explanation:** Re-examines entity store to confirm Sam's summary now includes founder detail—validating incremental enrichment.

In [65]:
conversation.predict(input="What do you know about Sam?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'Based on the information I have, I know that Sam is the founder of a company called Daimon.'

**Explanation:** Final targeted retrieval request: tests whether Sam's aggregated attributes (project + founder role) are surfaced coherently.

# About the Author

<div style="background-color: #f8f9fa; border-left: 5px solid #28a745; padding: 20px; margin-bottom: 20px; border-radius: 5px;">
  <h2 style="color: #28a745; margin-top: 0; font-family: 'Poppins', sans-serif;">Muhammad Atif Latif</h2>
  <p style="font-size: 16px; color: #495057;">Data Scientist & Machine Learning Engineer</p>
  
  <p style="font-size: 15px; color: #6c757d; margin-top: 15px;">
    Passionate about building AI solutions that solve real-world problems. Specialized in machine learning,
    deep learning, and data analytics with experience implementing production-ready models.
  </p>
</div>

## Connect With Me

<div style="display: flex; flex-wrap: wrap; gap: 10px; margin-top: 15px;">
  <a href="https://github.com/m-Atif-Latif" target="_blank">
    <img src="https://img.shields.io/badge/GitHub-Follow-212121?style=for-the-badge&logo=github" alt="GitHub">
  </a>
  <a href="https://www.kaggle.com/matiflatif" target="_blank">
    <img src="https://img.shields.io/badge/Kaggle-Profile-20BEFF?style=for-the-badge&logo=kaggle" alt="Kaggle">
  </a>
  <a href="https://www.linkedin.com/in/muhammad-atif-latif-13a171318" target="_blank">
    <img src="https://img.shields.io/badge/LinkedIn-Connect-0077B5?style=for-the-badge&logo=linkedin" alt="LinkedIn">
  </a>
  <a href="https://x.com/mianatif5867" target="_blank">
    <img src="https://img.shields.io/badge/Twitter-Follow-1DA1F2?style=for-the-badge&logo=twitter" alt="Twitter">
  </a>
  <a href="https://www.instagram.com/its_atif_ai/" target="_blank">
    <img src="https://img.shields.io/badge/Instagram-Follow-E4405F?style=for-the-badge&logo=instagram" alt="Instagram">
  </a>
  <a href="mailto:muhammadatiflatif67@gmail.com">
    <img src="https://img.shields.io/badge/Email-Contact-D14836?style=for-the-badge&logo=gmail" alt="Email">
  </a>
</div>

---

# ✅ Summary & Key Takeaways

This notebook compared three LangChain conversation memory strategies and demonstrated how they influence contextual reasoning and cost.

## Memory Types Covered
- ConversationBufferMemory: Stores full transcript (simple, high token growth).
- ConversationBufferWindowMemory: Retains only last k exchanges (bounded, may lose long-range facts).
- ConversationEntityMemory: Extracts & maintains evolving summaries per entity (rich personalization, extra LLM calls).

## When to Use What
| Scenario | Recommended Memory | Rationale |
|----------|--------------------|-----------|
| Short chat / debugging | Buffer | Simplicity > efficiency |
| Long sessions w/ limited budget | Window (k ~ 4–8) | Controls prompt size |
| Personalized assistants / multi-user facts | Entity | Structured recall of people/items |
| Hybrid (recent flow + persona facts) | Window + Entity | Combine breadth + depth |

## Practical Observations
- Window eviction is immediate: earlier enumeration (e.g., tips list) becomes unavailable once outside k.
- Entity memory scales by semantic summarization, not raw transcript length—useful for long-lived agents.
- `return_messages=True` enables tooling/prompt orchestration (function calling, inspection).
- Always sanitize secrets; environment variables > inline literals.

## Performance & Cost Notes
- Entity memory introduces additional LLM calls for extraction/summarization—cache aggressively in production.
- Monitor token drift: buffer memory can silently explode prompt size in user-driven loops.

## Extension Ideas
1. Add vector store for long-term semantic recall (RAG + memory fusion).
2. Persist entity store to Redis / DB for cross-session continuity.
3. Implement adaptive k (dynamic window size based on token budget).
4. Weight entity summaries with recency decay for evolving states.
5. Integrate tool-calling to enrich entity attributes (e.g., CRM lookup, calendar context).

## Suggested Next Steps
- Refactor repeated imports & consolidate setup at top.
- Wrap demos in utility functions + unit tests for regression safety.
- Add latency + token accounting instrumentation.

Feel free to adapt this structure across your other notebooks to maintain a consistent professional documentation standard.

---